Articolo per ulteriori spiegazioni:

https://medium.com/@kshitiz.sahay26/how-i-created-an-instruction-dataset-using-gpt-3-5-to-fine-tune-llama-2-for-news-classification-ed02fe41c81f

In [ ]:
import pandas as pd
import sklearn
from google.colab import drive
import nltk
import torch
from torch import nn
import numpy as np
from tqdm import tqdm
from torch.optim import Adam
from sklearn.model_selection import train_test_split

drive.mount('/content/gdrive')

# Dataset
# repository https://github.com/iresiragusa/NLP/tree/main
# https://www.kaggle.com/datasets/yufengdev/bbc-fulltext-and-category?select=bbc-text.csv
# scarichiamo il dataset e lo carichiamo su COLAB

root = "/content/gdrive/MyDrive/Colab Notebooks/torch/"
df = pd.read_csv(root+"data/BBC-text/bbc-text.csv")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [ ]:
# creaiamo l'istruzione per il llm (il prompt)
instruction = """Categorize the news article into one of the 5 categories:

business
politics
tech
sport
entertainment

"""

df['instruction'] = df.apply(lambda row: instruction, axis = 1)
df = df.rename(columns={"text": "input", "category": "output"})
df = df[['instruction', 'input', 'output']]

In [ ]:
# train-test split e ridenominiamo le colone
from sklearn.model_selection import train_test_split
(df_train, df_test) = train_test_split(df[['instruction', 'input', 'output']], test_size=0.2, random_state=17)
df_train = df_train.reset_index().drop('index', axis = 1)
df_test = df_test.reset_index().drop('index', axis = 1)

In [ ]:
df_train

,instruction,input,output
0,Categorize the news article into one of the 5 ...,singer ian brown in gig arrest former stone ...,entertainment
1,Categorize the news article into one of the 5 ...,eu ministers to mull jet fuel tax european uni...,business
2,Categorize the news article into one of the 5 ...,what price for trusted pc security you can ...,tech
3,Categorize the news article into one of the 5 ...,text messages aid disaster recovery text messa...,tech
4,Categorize the news article into one of the 5 ...,algeria hit by further gas riots algeria suffe...,business
...,...,...,...
1775,Categorize the news article into one of the 5 ...,bomb threat at bernabeu stadium spectators wer...,sport
1776,Categorize the news article into one of the 5 ...,big war games battle it out the arrival of new...,tech
1777,Categorize the news article into one of the 5 ...,berlin honours s korean director south korean ...,entertainment
1778,Categorize the news article into one of the 5 ...,doves soar to uk album summit manchester rock ...,entertainment


In [ ]:
# Saving as a CSV file
df_train.to_csv(f"{root}data/BBC-text/bbc-text-train.csv", index = False)
df_test.to_csv(f"{root}data/BBC-text/bbc-text-test.csv", index = False)